Solution for exercise
---------

```python
def message_func(edges):
    return {'m' : edges.src['h'] * edges.src['norm']}

reduce_func = fn.sum('m', 'h')

class GraphConv(gluon.Block):
    def __init__(self, in_feats, out_feats):
        super(GraphConv, self).__init__()
        self.linear = nn.Dense(out_feats)
    
    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # calculate norm = 1/sqrt(degree)
        norm = 1.0 / nd.sqrt(g.in_degrees(g.nodes()).astype('float32')).reshape(-1, 1)
        g.ndata['norm'] = norm
        # perform linear transformation
        h = self.linear(inputs)
        # set the node features
        g.ndata['h'] = h
        # trigger message passing, using the defined message_func and reduce_func.
        g.update_all(message_func, reduce_func)
        # get the result node features
        h = g.ndata.pop('h') * norm
        return h

inputs = nd.eye(34)  # featureless inputs
labeled_nodes = nd.array([0, 33])  # only the instructor and the president nodes are labeled
labels = nd.array([0, 1])  # their labels are different

net = GCN(34, 5, 2)
net.initialize()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.05})
loss_fn = gluon.loss.SoftmaxCELoss()

all_logits = []
for epoch in range(30):
    with autograd.record():
        logits = net(G, inputs)
        # we only compute loss for node 0 and node 33
        loss = loss_fn(logits[labeled_nodes], labels).sum()
    all_logits.append(logits.detach())
    
    loss.backward()
    trainer.step(batch_size=1)
    
    print('Epoch %d | Loss: %.4f' % (epoch, loss.asscalar()))
```